In [73]:
using Plots
gr()

Plots.GRBackend()

In [74]:
using MLDataPattern
using LIBSVM
using AxisArrays
using JuMP
using ProgressMeter

In [75]:
@showprogress 1 "foo" for i in 1:50
    sleep(0.1)
end

foo100% Time: 0:00:05


In [76]:
include("main.jl")

c

In [77]:
double_integrator = c.MPC.discretize(c.MPC.CTLinearSytstem([0. 1; 0 0], [0. 1]'), 0.1)
time = Axis{:time}(linspace(0, 9 * double_integrator.Δt, 10))
side = Axis{:side}([:left, :right])
model = c.MPC.create_model(double_integrator, time, side);
numresults = 10000
results = c.Record[]
@showprogress 1 "gathering data... " for i in 1:numresults
    push!(results, c.Record(model))
end

gathering data... 100%  ETA: 0:00:01

In [78]:
examples = convert(Vector{c.Example}, results);

labels = [e.best_action for e in examples]
data = hcat([vcat(e.contact, e.duals) for e in examples]...);

In [79]:
(data_train, labels_train), (data_test, labels_test) = splitobs((data, labels), 0.5);

In [80]:
@time svm = svmtrain(data_train, labels_train; cost=100., probability=true)
@time predicted_labels, decision_values = svmpredict(svm, data_test)

 13.726812 seconds (4.58 k allocations: 13.288 MB, 0.04% gc time)
  3.878712 seconds (4.52 k allocations: 22.124 MB, 0.09% gc time)


([2,16,2,13,1,18,3,1,3,2  …  15,11,9,7,2,1,3,14,4,1],
[0.00311711 0.0672858 … 0.00419944 0.7895; 0.0043126 0.000891773 … 0.00516554 0.00180899; … ; 0.000254341 0.00127563 … 0.00295882 0.000619531; 0.000178945 0.000960092 … 0.00175041 0.00044275])

In [81]:
mean(predicted_labels .== labels_test)

0.6514

In [96]:
q0 = 2 * rand() - 1
v0 = 4 * rand() - 2
qlimb0 = 2 * rand() - 1
state = c.MPC.State(q0, v0, qlimb0)
contact = AxisArray(rand(Bool, 2, length(time)), side, time)

2-dimensional AxisArray{Bool,2,...} with axes:
    :side, Symbol[:left,:right]
    :time, linspace(0.0,0.9,10)
And data, a 2×10 Array{Bool,2}:
  true   true   true   true  false  false  false  true  true  false
 false  false  false  false   true   true   true  true  true   true

In [97]:
function svmbeam(svm::LIBSVM.SVM, model::c.MPC.MPCModel, state::c.MPC.State, contact, random=false)
    active_set = [BitArray(contact)]
    objectives = Float64[]
    for i in 1:100
        new_active_set = Tuple{Float64, BitArray{2}}[]
        for contact in active_set
            result = c.MPC.solve!(model, state, contact_sequence=contact)
            if result.status != :Optimal
                result = c.MPC.solve!(model, state, contact_sequence=contact, relax=true)
            end
            push!(objectives, getvalue(getobjective(result.model.m)))
            features = vcat(vec(contact), result.model.m.linconstrDuals)
            _, decision_values = svmpredict(svm, reshape(features, length(features), 1))
            if random
                actions = svm.labels[randperm(length(svm.labels))]
            else
                actions = svm.labels[reverse(sortperm(vec(decision_values)))]
            end
            for action in actions[1:5]
                contact = copy(contact)
                contact[action] = !contact[action]
                result = c.MPC.solve!(model, state, contact_sequence=contact)
                if result.status != :Optimal
                    result = c.MPC.solve!(model, state, contact_sequence=contact, relax=true)
                end
                push!(new_active_set, (getvalue(getobjective(result.model.m)), BitArray(contact)))
            end
        end
        active_set = [s[2] for s in sort(new_active_set)[1:5]]
    end
    objectives
end
                

svmbeam (generic function with 3 methods)

In [102]:
plot(svmbeam(svm, model, state, contact))
savefig("svmbeam.svg")

In [104]:
plot(svmbeam(svm, model, state, contact, true))
savefig("svmbeam_random.svg")

In [100]:
function svmsolve(svm::LIBSVM.SVM, model::c.MPC.MPCModel, state::c.MPC.State, contact)
    visited = Set([BitArray(contact)])
    objectives = Float64[]
    for i in 1:100
        result = c.MPC.solve!(model, state, contact_sequence=contact)
        if result.status != :Optimal
            result = c.MPC.solve!(model, state, contact_sequence=contact, relax=true)
        end
        push!(objectives, getvalue(getobjective(result.model.m)))

        features = vcat(vec(contact), result.model.m.linconstrDuals)
        _, decision_values = svmpredict(svm, reshape(features, length(features), 1))
        actions = svm.labels[reverse(sortperm(vec(decision_values)))]
        for action in actions
            contact[action] = !contact[action]
            b = BitArray(contact)
            if !(b in visited)
                push!(visited, b)
                break
            else
                contact[action] = !contact[action]  # put it back
            end
        end
    end
    objectives
end

svmsolve (generic function with 3 methods)

In [101]:
plot(svmsolve(svm, model, state, contact))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 30 
 
 
 32 
 
 
 34 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 21.7148,269.048 27.5163,269.447 33.3178,194.963 39.1193,165.976 44.9208,91.3295 50.7223,133.377 56.5238,180.857 62.3253,174.693 68.1268,133.701 73.9283,140.554 
 79.7298,91.3072 85.5313,48.0762 91.3328,118.376 97.1342,148.85 102.936,224.151 108.737,223.445 114.539,225.959 120.34,227.566 126.142,169.374 131.943,131.171 
 137.745,106.839 143.546,161.597 149.348,189.821 155.149,176.461 160.951,212.611 166.752,224.619 172.554,197.619 178.355,144.644 184.157,174.81 189.958,210.202 
 195.76,270.71 201.561,269.918 207.363,196.858 213.164,201.057 218.966,155.627 224.767,111.684 230.569,102.612 236.37,160.502 242.172,184.851 247.973,168.824 
 253.775,145.721 259.576,162.689 265.378,136.431 271.179,77.4446 276.981,85.0654 282.782,130.817 288.584,176.757 294.385,171.626 300.187,160.247 305.988,162.275 
 311.79,92.9544 317.591,58.2494 323.393,3.93701 329.194,50.283 334.996,98.1093 340.797,89.2715 346.599,115.842 352.4,124.19 358.202,78.4931 364.003,32.9091 
 369.805,88.4555 375.606,122.198 381.408,189.609 387.209,187.921 393.011,302.715 398.812,304.412 404.614,237.076 410.415,203.306 416.217,147.891 422.018,193.497 
 427.82,239.149 433.621,230.787 439.423,205.232 445.224,214.08 451.026,166.285 456.827,119.923 462.629,174.146 468.43,208.87 474.232,278.139 480.033,276.104 
 485.835,287.534 491.636,292.684 497.438,246.826 503.239,201.037 509.041,193.512 514.842,252.522 520.644,278.743 526.445,261.755 532.247,283.869 538.048,299.924 
 543.85,275.629 549.651,217.706 555.453,226.634 561.254,270.63 567.056,315.941 572.857,311.717 578.659,384.144 584.46,384.952 590.261,324.643 596.063,289.18 
 
 "/>
 
 
 
 
 y1

In [88]:
function topn(n)
    [svm.labels[reverse(sortperm(decision_values[:, i]))[1:n]] for i in 1:size(decision_values, 2)]
end

topn (generic function with 1 method)

In [89]:
mean(getindex.(topn(1), 1) .== labels_test)

0.6514

In [90]:
t = topn(5)
mean([labels_test[i] in t[i] for i in 1:length(labels_test)])

0.9802

In [91]:
histogram(labels)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 0 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [92]:
histogram(predicted_labels)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1